In [2]:
import cv2
import numpy as np
import pyttsx3

# Load the pre-trained YOLO object detection model
net = cv2.dnn.readNet(r"C:\Users\SR_Ravi\Desktop\ravi\yolov3.weights", r"C:\Users\SR_Ravi\Desktop\ravi\yolov3.cfg")

# Load the classes of objects that the model can recognize
with open(r"C:\Users\SR_Ravi\Desktop\ravi\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set up the text-to-speech engine
engine = pyttsx3.init()

# Open the camera for live video input
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Convert the frame to a blob for input to the YOLO model
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), (0, 0, 0), True, crop=False)
    
    # Set the input to the YOLO model and get the output
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())
    
    # Process the output of the YOLO model to find object detections
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])
                x = center_x - w//2
                y = center_y - h//2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    
    # Apply non-maximum suppression to remove overlapping detections
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Draw bounding boxes and labels around the detected objects
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        label = classes[class_ids[i]]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        engine.say(label)
        engine.runAndWait()
    
    # Display the output frame with object detections
    cv2.imshow("Object Detection", frame)
    
    # Check for user input to quit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
